In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

Step 1: Get the list of CFIDs

In [31]:
url = 'https://mfp.mpfa.org.hk/eng/mpp_list.jsp'
response = requests.get(url)


In [33]:
soup = BeautifulSoup(response.text, 'html.parser')

In [37]:
# <input type="button" value="More" class="btn_more" cfid="1520">
more_buttons = soup.find_all("input", {"class": "btn_more"})

In [38]:
cfid_list = []
for btn in more_buttons:
    cfid_list.append(btn['cfid'])

In [40]:
len(cfid_list)

475

Step 2: Send a request to each detail page and combine them

In [55]:
def response2df(response):
    df = pd.read_html(response.text)
    df2 = df[-2]
    df3 = pd.concat([df2.loc[2:10], df2.loc[13:23], df2.loc[26:41]])
    df4 = df3.T
    df4.columns = df4.iloc[0]
    df4.drop(df4.index[0], inplace=True)
    return df4

In [56]:
url = 'https://mfp.mpfa.org.hk/eng/cf_detail.jsp?cf_id=1520'
response = requests.get(url)

In [57]:
response2df(response)

,Name of MPF trustee,Name of MPF scheme,Name of the constituent fund,Fund Class,Fund Type,Fund Type - Full Descriptor,Fund size (HKD Million),Launch Date,Financial Period End Date,Annualized Return / Cumulative Return (1 Year) (as at 31 August 2023),...,Joining Fee,Annual Fee,Contribution Charge,Bid Spread,Offer Spread,Withdrawal Charge,Other Expenses,On-going Cost Illustration (OCI) – 1 Year,On-going Cost Illustration (OCI) – 3 Year,On-going Cost Illustration (OCI) – 5 Year
1,AIA Company (Trustee) Limited,AIA MPF - Prime Value Choice,Age 65 Plus Fund,n.a.,Mixed Assets Fund - Default Investment Strateg...,Mixed Assets Fund - Global - Maximum equity 25%,"1,553.48 (as at 31 August 2023)",1 Apr 2017,30 Nov,-0.03% p.a. / -0.03%,...,0%,0%,0%,0%,0%,0%,A range of other expenses may be chargeable to...,HKD 8,HKD 26,HKD 45


In [66]:
df_combined = pd.DataFrame()
for cfid in cfid_list:
    print(cfid)
    url = 'https://mfp.mpfa.org.hk/eng/cf_detail.jsp?cf_id=' + str(cfid)
    response = requests.get(url)
    df = response2df(response)
    df['cfid'] = cfid
    df.set_index('cfid', inplace=True)
    df_combined = pd.concat([df_combined, df])
    time.sleep(1)
    

1520
878
875
137
132
131
1579
1519
876
145
657
102
141
133
146
877
699
134
142
658
772
770
769
773
771
1530
778
1529
779
782
780
781
775
776
777
774
1522
210
207
697
1521
208
830
831
206
653
798
211
1979
209
1532
178
1531
180
181
174
175
173
177
176
179
1099
1534
187
1859
659
1533
188
185
184
701
660
189
183
190
909
800
799
182
1100
702
703
705
707
710
1819
1820
1821
908
1536
899
212
1535
812
215
811
213
913
214
900
901
1538
221
216
225
1537
814
219
813
218
224
220
217
222
896
912
223
897
898
1540
1044
1041
1539
1048
1043
1045
1040
1047
1039
1042
1542
234
228
233
654
1541
999
1001
231
227
232
1959
226
230
1119
1000
229
1546
236
1545
1839
237
895
603
238
606
1840
1552
2079
278
280
272
1551
2080
1159
704
706
708
709
711
1419
1420
275
268
276
273
274
1799
1439
271
270
1555
1556
283
282
1554
1553
286
287
290
291
285
284
288
289
806
303
310
808
304
307
805
1625
311
305
1626
302
309
306
308
807
1627
1628
1629
1630
802
324
322
804
318
326
801
1619
317
320
1620
325
321
319
323
803
1621
1622
16

In [69]:
df_combined.head()

,Name of MPF trustee,Name of MPF scheme,Name of the constituent fund,Fund Class,Fund Type,Fund Type - Full Descriptor,Fund size (HKD Million),Launch Date,Financial Period End Date,Annualized Return / Cumulative Return (1 Year) (as at 31 August 2023),...,Joining Fee,Annual Fee,Contribution Charge,Bid Spread,Offer Spread,Withdrawal Charge,Other Expenses,On-going Cost Illustration (OCI) – 1 Year,On-going Cost Illustration (OCI) – 3 Year,On-going Cost Illustration (OCI) – 5 Year
cfid,,,,,,,,,,,,,,,,,,,,,
1520,AIA Company (Trustee) Limited,AIA MPF - Prime Value Choice,Age 65 Plus Fund,n.a.,Mixed Assets Fund - Default Investment Strateg...,Mixed Assets Fund - Global - Maximum equity 25%,"1,553.48 (as at 31 August 2023)",1 Apr 2017,30 Nov,-0.03% p.a. / -0.03%,...,0%,0%,0%,0%,0%,0%,A range of other expenses may be chargeable to...,HKD 8,HKD 26,HKD 45
878,AIA Company (Trustee) Limited,AIA MPF - Prime Value Choice,American Fund,n.a.,Equity Fund - United States Equity Fund,Equity Fund - North America,"3,289.82 (as at 31 August 2023)",23 Sep 2011,30 Nov,+13.92% p.a. / +13.92%,...,0%,0%,0%,0%,0%,0%,A range of other expenses may be chargeable to...,HKD 9,HKD 27,HKD 47
875,AIA Company (Trustee) Limited,AIA MPF - Prime Value Choice,Asian Bond Fund,n.a.,Bond Fund - Asia Bond Fund,Bond Fund - Asia-Pacific,"1,371.74 (as at 31 August 2023)",23 Sep 2011,30 Nov,+1.36% p.a. / +1.36%,...,0%,0%,0%,0%,0%,0%,A range of other expenses may be chargeable to...,HKD 8,HKD 25,HKD 44
137,AIA Company (Trustee) Limited,AIA MPF - Prime Value Choice,Asian Equity Fund,n.a.,Equity Fund - Asia Equity Fund,Equity Fund - Asia-Pacific,"6,546.10 (as at 31 August 2023)",1 Dec 2004,30 Nov,+6.42% p.a. / +6.42%,...,0%,0%,0%,0%,0%,0%,A range of other expenses may be chargeable to...,HKD 18,HKD 55,HKD 94
132,AIA Company (Trustee) Limited,AIA MPF - Prime Value Choice,Balanced Portfolio,n.a.,Mixed Assets Fund - 41% to 60% Equity,Mixed Assets Fund - Global - Maximum equity ar...,"7,366.57 (as at 31 August 2023)",1 Dec 2000,30 Nov,+1.39% p.a. / +1.39%,...,0%,0%,0%,0%,0%,0%,A range of other expenses may be chargeable to...,HKD 17,HKD 54,HKD 93


In [68]:
df_combined.to_excel('MPF_details.xlsx')